###Инструкция

1. Авторизируйтесь в своем Google Account'e
2. Зайдите в Google Cloud Console
3. Выберите существующий Google Cloud project или создайте новый
4. Включите в Google Cloud мени APIs and Servicies - Google BigQuery API для выбранного проекта
5. Скопируйте этот Colab Notebook на свой диск
6. Пройдите [по данной ссылке](https://cloud.google.com/bigquery/docs/sandbox), нажмите в секции **Getting started with the sandbox**, п.1 на кнопку **Go to BigQuery Sandbox**  - в противном случае, Google будет требовать оформления триалки. Используйте Sandbox для первого задания.
7. Выполните все задания в своем скопированном Notebook
8. Пошарьте нам свои результаты

In [ ]:
#run this cell first
import pandas as pd
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


##Задание № 1
Данные по GA360 лежат в Google BigQuery (схема —
https://support.google.com/analytics/answer/3437719). 
На ЦИАН часто приходят парсеры. 

1.   Напишите запрос, который будет выводить MAU с учетом того, что ботов надо исключить
2.   На основе каких параметров вы бы предложили определять
ботов?

###Ответ

###Таблица
Данные за 6 месяцев (для наглядности)
```
bigquery-public-data.google_analytics_sample.ga_sessions_*
WHERE _TABLE_SUFFIX BETWEEN '20170101' AND '20170631
```

###Введение
Первым, что пришло в голову, это использовать разницу между хитами в миллисекундах (колонка hits.time). Робот должен по идее парсить сайт очень быстро, что невозможно повторить человеку. А если это именно парсер, его алгоритм работы по идее очень простой (например, нет искусственной неактивности в течение времени как когда-то у кликеров). 

###Трудности
Разница между хитами по всем юзерам представляет собой ассиметричное распределение (большинство значений находятся около нуля). Близкие к нулю значения (то есть разница между хитами в мил.секундах крайне мала) принимается за аномалию. Но хиты могут быть совершенно разными, не только клик, они могут следовать друг за другом почти мгновенно, то есть появляться в сессии, осуществляемой человек, поэтому разница близкая к нулу не всегда свидетельствует о том, что это аномалия (выброс). Также нет возможности исключить аномалии через стандартные алгоритмы работы с выбросами, например IQR, потому формально это как раз будет часть распределения.

###Решение
Надо смотреть разницу от хита к хиту в разрезе каждого посетителя, сравнивания каждую из них с медианным значением всех разниц (в примере = 9595 мил.секунд или 9.5 секунды): больше медианы - норма, меньше - аномалия. Далее вычислить соотношение по каждому посетителю кол-ва нормальных разниц (выглядит как действия человека) к общеку кол-ву хитов, исключая самый первый (там всегда разница нулевая). 

Получившееся соотношение сравнить с пороговым, принятым за 0.05: то есть только 5% от всех хитов абстрактного посетителя были сделаны с такой разницей, которые могли бы считаться нормальными (свойственными человеку). Все, что меньше порога - это id ботов, которые мы будем исключать из подсчета MAU, где MAU количество уников за месяц.

###Дальнейшие возможные шаги
1.   Исследовать посетителей, маркированных как боты, по схеме: провалидировать, действительно ли это боты (например, обогатить эти данные с данными по внутренними таблицами, может, у нас тоже логи есть свои) + возможно, у них есть общие параметры по схеме, которые можно также использовать для фильтра (гео, IP пр.). 
Провалидировал, есть странные посетители -  есть всего один хит после захода. Допускаю, что это могли бы быть юзеры. Отсюда надо изучить более подробно как работают парсеры (является ли это их паттерном), оставил в таблице bots закомменчанное условие (WHERE total_hits > 1).
2.   (Со скепсисом пишу) После валидации можно пролейбировать посетителей (бот, человек), отобрать фичи и сделать простенький классификатор в рамках BQ. Сравнить с имеющимся фильтром в плане точности классификации. Но так в начале сказал, что алгоритм парсера предполагается довольно простым, не факт, что модель будет лучше работать, чем захардкоденные уточненные тривиальные правила для фильтра + стоит ли это траты времени, которое выражается не только в отборе фичей, обучении, но и траты ресуросов на выполнение, переобучение, поддержание (считаю, что оставлять надо хорошее легаси, а не франкенштейнов) пр. 





In [ ]:
query = '''
--порог, меньше которого активность посетителя будет принята за активность парсера
DECLARE threshold FLOAT64 DEFAULT 0.05;

WITH stats AS (SELECT
                    *,
                    IF(LAG(milsec) OVER (PARTITION BY fullVisitorId ORDER BY milsec) IS NULL, 0, LAG(milsec) OVER (PARTITION BY fullVisitorId ORDER BY milsec)) AS prev_hit_time,
                    IF(LAG(milsec) OVER (PARTITION BY fullVisitorId ORDER BY milsec) IS NULL, 0,
                    ROUND(IF(milsec - LAG(milsec) OVER (PARTITION BY fullVisitorId ORDER BY milsec) = milsec, milsec, milsec - LAG(milsec) OVER (PARTITION BY fullVisitorId ORDER BY milsec)), 3)) AS diff
               FROM
                (SELECT 
                        fullVisitorId,
                        --форматируем дату в YYYY-MM
                        FORMAT_DATE('%Y-%m', PARSE_DATE('%Y%m%d', date)) AS month,
                        --hits.time - the number of milliseconds after the visitStartTime when this hit was registered. The first hit has a hits.time of 0
                        hit.time AS milsec,
                    FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`, UNNEST(hits) as hit
                    WHERE _TABLE_SUFFIX BETWEEN '20170101' AND '20170631')),
    --медианное значение разницы между хитами
    median AS (SELECT
                    APPROX_QUANTILES(diff, 100)[OFFSET(50)] AS median
               FROM
                stats
               WHERE milsec != 0.0 AND prev_hit_time != 0.0),

    --исключая первую запись, она по дефолту в мил.секундах нулевая с момента старта сессии
    --это также исключает тех, кто зашел и сразу вышел (не было больше хитов)
    --они могут быть как ботами, там и людьми, нет возможности определетить, кто есть кто в таком случае по существующей схеме
    bots AS (SELECT
                fullVisitorId
             FROM
            (SELECT 
                DISTINCT 
                fullVisitorId,
                ROUND(SUM(higher_than_median) OVER (PARTITION BY fullVisitorId)/total_hits, 4) AS normality_rate
             FROM
                (SELECT 
                    * EXCEPT(month, milsec, prev_hit_time, diff),
                    --исключая первую запись, она по дефолту в мил.секундах нулевая с момента старта сессии
                    COUNT(fullVisitorId) OVER (PARTITION BY fullVisitorId) AS total_hits,
                    CAST(diff > (SELECT * FROM median) AS INT64) AS higher_than_median
                FROM stats
                    WHERE milsec != 0.0)
             --формально есть еще хит, но он по дефолту нулевой с момента старта
             --он исключен в подзапросе, поэтому в total_hits не учитывается 
             --WHERE total_hits > 1
             )
             WHERE normality_rate < threshold)
SELECT
    month,
    COUNT(DISTINCT fullVisitorId) AS MAU
FROM stats
WHERE fullVisitorId NOT IN (SELECT * FROM bots)
GROUP BY month
ORDER BY month
'''

#свой project_id в Google Cloud
project_id = 'cian-311410'

df = pd.io.gbq.read_gbq(query, project_id=project_id, verbose=False, dialect='standard')
df.head(6)

,month,MAU
0,2017-01,50892
1,2017-02,49168
2,2017-03,55499
3,2017-04,53407
4,2017-05,50154
5,2017-06,49893


## Задание № 2
Представьте, что у вас есть правильная игровая кость с шестью гранями. Вы
бросаете ее N раз. Напишите, пожалуйста, функцию на python, принимающую на
входе два значения:

  1.   N - количество бросков кубика
  2.   S - сумма значений, выпавших на кубике
 
Функция на выходе должна отдавать вероятность выпадения суммы значений
кубика S при N бросках.

###Ответ

In [ ]:
#обработка неверных типов данных etc.
def exceptions(value, var_name = 'variable'):
  if type(value) is str:
    try:
      print("{} is string type. Converting...".format(var_name))
      if (int(float(value)) == float(value)):
        value = int(value)
      else:
        value = float(value)
    except:
      raise TypeError("Only numbers are allowed. Please, change value of {}".format(var_name))
  if type(value) is float:
      value = int(value)
      print("{} is float type. Only integer is allowed (number of throws is discete).".format(var_name))
      print("Making integer instead... Now {} is {}".format(var_name, value))
  if type(value) is not int:
    raise Exception("{} is {}. Cannot use this type of data for that function. Use positive integer instead.".format(var_name, type(value)))
  
  if (var_name == 'N' and value < 0):
    raise Exception("{} is {}. Cannot make a throw reversed in time. It's not a Tenet movie, sorry :(. Please, provide a positive number.".format(var_name, value))
  if (var_name == 'N' and value == 0):
    raise Exception("{} is 0. Cannot throw a dice zero times, sorry. Please, prodive a positive number.".format(var_name))
  if (var_name == 'S' and value < 0):
    raise Exception("{} is {}. Total sum can not be negative! Please, provide a positive number.".format(var_name, value))
  return value

#желаемая функция
def probability_sum(n=1, s=1):
  probability = 0.0
  n = exceptions(n, 'N')
  s = exceptions(s, 'S')

  if (n != 0 and s == 0):
    raise Exception("S is 0. Total sum can not be zero GIVEN N > 0")
  if (n == 0 and s == 0):
    print("Probability of S = 0 GIVEN N = 0 equals to 1")
    probability = 1.0
    return probability

  if n == 1:
    print("Probalitity of S = {} GIVEN N = 1 equals to 1/6".format(s))
    probability = 0.1666
    return probability

  #самое простое, отсекаем достоверное и невозможное события
  #сумма меньше минимального порога суммы после N бросков (всегда достоверное)
  #сумма больше максимального порога суммы после N бросков (всегда невозможное)
  #например, 2 броска, минимальный порог - 2 (1 и 1), максимальный - 12 (6 и 6).
  min_value = 1
  max_value = 6
  min_range = min_value * n
  max_range = max_value * n
  if s < min_range:
    print("Probability of S = {} GIVEN {} throws equals to 1, because result of {} throws is always not less than {}".format(s, n, n, min_range))
    probability = 1.0
    return probability

  elif s > max_range:
    print("Probability of S = {} GIVEN {} throws equals to 0, because result of {} throws is always not more than {}".format(s, n, n, max_range))
    probability = 0.0
    return probability
  
  #таблица для вероятностей
  #shape зависит от числа бросков (n)
  #например, n = 2 - это 2d array c 2 элементами-массивами, 
  #где кол-во элементов в каждом = вероятностый диапазон возможных сумм после 2-х росков, то есть 12
  #в первом - будут вероятности значений одного броска, 
  #во втором - веротности сумм двух бросков
  #при n = 3 - в 3-м массиве - вероятности сумм трех бросков, а элементов каждом из массивов - 18
  prob_table = [[ 0 for j in range(n * 6)] for i in range(n)]

  #вероятность выпадение каждой грани кубика при первом броске
  for value in range(6):
        prob_table[0][value] = 1 / 6
  
  #тут самое веселоe
  #Использовано динамическое программирование
  #разбиваем задачи на подзадачи, получившиеся данные в одной подзадаче используем в другой
  #Вычисления вероятностей
  #Для всех значений при броске от 2х раз включительно
  #i пробегается по n массивам с вероятностями сумм, где n - броски
  for i in range(1, n):
      #j пробегается по вероятностям всех возможных сумм, начиная с первого массива
      for j in range(len(prob_table[i-1])):
          #раз за разом обновляем массив i, сохраняя для следующего массива i+1 данные.
          for k in range(6):
              if (prob_table[i-1][j] != 0 and prob_table[i-1][k] != 0):
                  prob_table[i][j + k] += (prob_table[i-1][j] * prob_table[0][k])

  for i in range(len(prob_table[n-1])):
        if i+n == s:
          probability = prob_table[n-1][i]
          print("Probability of S = %d GIVEN N = %d equals to %0.4f" % (s, n, prob_table[n-1][i]))
          break

  return probability

In [ ]:
#тест
probability = probability_sum(3, 17)
probability

Probability of S = 17 GIVEN N = 3 equals to 0.0139


0.013888888888888888

## Задание № 3
У вас есть три вертикали, с которыми вы работаете:

1.   Оценка квартиры (https://www.cian.ru/kalkulator-nedvizhimosti/)
2.   Ипотека (https://www.cian.ru/ipoteka-main/)
3.   Журнал (https://www.cian.ru/magazine/)

Представьте, что вас в команде 2 аналитика. Как бы вы предложили распределить
свои усилия по этим проектам? На какие факторы обратили бы внимание и почему

###Ответ

Распределение усилий зависит от следующих факторов:

1. Команда: надо понять, какие у нас скиллы "в среднем" на команду. Условно, если мы оба джуна - команда Джуниор; далее Полумидл (кто-то из нас джун, кто-то мидл), Мидл (оба мидла, один джун, один синиор; наоборот) etc. От этого + от сложности каждого проекта (п.3) зависит сможем ли мы работать на всех трех направлениях, хватает ли компененций, не нужели нам ментор или еще один человек в команду для усиления, справляемся ли мы по срокам. Плюс по возможности я бы постарался учесть пожелание коллеги (в чем бы он хотел развиваться), и если складывается, дать ему ту вертикаль (или задачи по вертикалям), которые его максимально вдохновляют/дают рост. Это (адекватная нагрузка на человека + учет интересов), помимо прочего, еще и страховка от того, что человек может перегореть, устать от проектов, и, как следствие, уйти.

2. Приоритеты бизнеса:
Тулзы и блог (журнал) не ради тулзов и блога сделаны. Они несут value потребителю, а это в конечном итоге вносит вклад в выручку. Вопрос в том, на что делается сейчас основная ставка: что дает (= оптимизация этого) или может дать (= точка роста) наибольший value. От этого будет идти приоритизация тасков в спринте, бэклоге.

3. Сложность проектов: основная ставка бизнеса не означает то, что таски по самому приоритетному будут самые сложные. Они могут быть самыми важными, что не равно самому сложному в реализации. Сложность зависит от технических требований к проекту (и, конечно, соответствия навыков команды этим требованиям) и выделенного времени на проект. Например, высокие требования + мало времени = высокая сложность. Необходимо проанализировать сложность работы с каждой вертекалью в зависимости от планов owner'ов каждой из них.

4. Внешние факторы в лице внутренних заказчиков: требовать могут что-то еще хоть вчера, но реально ознакомится с выполненной работой смогут не ранее, скажем, чем через неделю. Имея эту информацию, можно корректировать сроки заказчиков не вопросом, когда надо, чтобы было сделано, а когда он, заказчик, сможет реально внимательно ознакомится с тем, что ему будет от нас предоставлено. Так у команды будет возможности более гибко выстраивать спринты. Косвенно зависит от того, как умеет коммуницировать команда, то есть хорошо бы знать и учитывать софт-скиллы команды.